<a href="https://colab.research.google.com/github/HackerJacky/TimeSeriesColab/blob/main/%E6%99%82%E9%96%93%E5%BA%8F%E5%88%97%E5%88%86%E6%9E%90_HW4_%E5%8F%83%E8%80%83%E8%B3%87%E6%96%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

好的，這是以實際處理資料的練習為主要題型的題組，著重於使用 S&P 500 指數月報酬建立 AR 模型並進行預測能力評估。

**題組：S&P 500 指數月報酬之 AR 模型預測能力評估**

**背景：**

我們希望利用 2010 年至今的 S&P 500 指數月報酬資料，建立時間序列模型來預測未來的月報酬。我們將比較 AR(1) 和 AR(2) 這兩個模型，並使用 Diebold-Mariano (DM) 檢定來判斷哪個模型具有更好的預測能力。

**資料：**

我們使用 `yfinance` 套件下載 2010-01-01 至 2024-12-31 的 S&P 500 指數資料 (`^GSPC`)。

**問題：**

1.  **資料處理：**
    * (a)  請撰寫 Python 程式碼，完成以下步驟：
        * 下載 2010-01-01 至 2024-12-31 的 S&P 500 指數資料 (`^GSPC`)，僅保留調整後收盤價 (`Adj Close`)。
        * 將資料轉換為月頻率，並計算月報酬率。
        * 建立月報酬率的落後一期 (`ret_sp500_1`) 和落後二期 (`ret_sp500_2`)。
        * 印出前 5 筆處理後的資料，並顯示資料的資訊 (`.info()`)。
    * (b)  為什麼我們需要計算報酬率，而不是直接使用股價進行建模？

2.  **模型建立與樣本劃分：**
    * (a)  請撰寫 Python 程式碼，完成以下步驟：
        * 使用前 80% 的資料建立 AR(1) 模型和 AR(2) 模型。
        * 印出 AR(1) 模型和 AR(2) 模型的模型摘要 (`.summary()`)。
    * (b)  解釋 AR(1) 模型和 AR(2) 模型的方程式。

3.  **樣本外預測與績效衡量：**
    * (a)  請撰寫 Python 程式碼，計算 AR(1) 模型和 AR(2) 模型在剩餘 20% 資料上的樣本外均方誤差 (MSE)。
    * (b)  比較兩個模型的樣本外 MSE，哪個模型的預測能力較好？

4.  **Diebold-Mariano 檢定：**
    * (a)  請撰寫 Python 程式碼，對 AR(1) 和 AR(2) 模型的樣本外預測誤差進行 Diebold-Mariano 檢定。
    * (b)  解釋 DM 檢定的結果（檢定統計量、P 值）。
    * (c)  根據 DM 檢定的結果，我們對 AR(1) 和 AR(2) 模型的預測能力有什麼結論？

**參考答案：**

1.  **資料處理：**

    * (a)  Python 程式碼：

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 下載資料
data = yf.download('^GSPC', start='2010-01-01', end='2024-12-31')
data = data[['Adj Close']]
data.columns = ['price_sp500']
data.index = pd.to_datetime(data.index)

# 轉換為月頻率並計算月報酬率
data_m = data.resample('M').last().copy()
data_m['ret_sp500'] = np.log(data_m.price_sp500).diff()

# 建立落後期
data_m['ret_sp500_1'] = data_m.ret_sp500.shift(1)
data_m['ret_sp500_2'] = data_m.ret_sp500.shift(2)

# 印出前 5 筆資料和資料資訊
print(data_m.head())
print(data_m.info())

* (b)  我們需要計算報酬率，而不是直接使用股價進行建模，主要有以下原因：
        * **平穩性 (Stationarity):** 報酬率通常比股價更接近平穩的時間序列。許多時間序列模型（如 AR 模型）都假設資料是平穩的。
        * **相對性：** 報酬率表示的是價格變動的百分比，更能反映投資的收益或損失，具有相對性，方便比較不同資產之間的表現。
        * **經濟意義：** 報酬率具有更直接的經濟意義，反映了投資的回報。

2.  **模型建立與樣本劃分：**

    * (a)  Python 程式碼：

In [ ]:
# 樣本劃分
train_size = int(len(data_m) * 0.8)
train_data = data_m.iloc[:train_size].copy()

# 建立 AR(1) 模型
y_ar1 = train_data['ret_sp500'].dropna()
x_ar1 = sm.add_constant(train_data['ret_sp500_1'].dropna())
x_ar1 = x_ar1.loc[y_ar1.index]  # 確保索引一致
model_ar1 = sm.OLS(y_ar1, x_ar1).fit()
print("AR(1) Model Summary:")
print(model_ar1.summary())

# 建立 AR(2) 模型
y_ar2 = train_data['ret_sp500'].dropna()
x_ar2 = sm.add_constant(train_data[['ret_sp500_1', 'ret_sp500_2']].dropna())
x_ar2 = x_ar2.loc[y_ar2.index]  # 確保索引一致
model_ar2 = sm.OLS(y_ar2, x_ar2).fit()
print("\nAR(2) Model Summary:")
print(model_ar2.summary())

* (b)  AR(1) 模型方程式：
        
        ```
        ret_t = \phi_0 + \phi_1 * ret_{t-1} + \epsilon_t
        ```
        
        其中 \(ret_t\) 是當期報酬率，\(ret_{t-1}\) 是前一期報酬率，\(\phi_0\) 是常數項，\(\phi_1\) 是係數，\(\epsilon_t\) 是誤差項。
        
        AR(2) 模型方程式：
        
        ```
        ret_t = \phi_0 + \phi_1 * ret_{t-1} + \phi_2 * ret_{t-2} + \epsilon_t
        ```
        
        其中 \(ret_t\) 是當期報酬率，\(ret_{t-1}\) 是前一期報酬率，\(ret_{t-2}\) 是前兩期報酬率，\(\phi_0\) 是常數項，\(\phi_1\) 和 \(\phi_2\) 是係數，\(\epsilon_t\) 是誤差項。

3.  **樣本外預測與績效衡量：**

    * (a)  Python 程式碼：

In [ ]:
# 樣本外預測
test_data = data_m.iloc[train_size:].copy()

# AR(1) 樣本外預測
ar1_forecasts = model_ar1.params['const'] + model_ar1.params['ret_sp500_1'] * test_data['ret_sp500_1'].dropna()
ar1_errors = test_data['ret_sp500'].dropna() - ar1_forecasts.loc[test_data['ret_sp500'].dropna().index]
mse_ar1 = np.mean(ar1_errors**2)
print(f"AR(1) Out-of-Sample MSE: {mse_ar1}")

# AR(2) 樣本外預測
ar2_forecasts = model_ar2.params['const'] + model_ar2.params['ret_sp500_1'] * test_data['ret_sp500_1'].dropna() + model_ar2.params['ret_sp500_2'] * test_data['ret_sp500_2'].dropna()
ar2_errors = test_data['ret_sp500'].dropna() - ar2_forecasts.loc[test_data['ret_sp500'].dropna().index]
mse_ar2 = np.mean(ar2_errors**2)
print(f"AR(2) Out-of-Sample MSE: {mse_ar2}")

* (b)  比較 `mse_ar1` 和 `mse_ar2` 的值。MSE 較小的模型預測能力較好。

4.  **Diebold-Mariano 檢定：**

    * (a)  Python 程式碼：

In [ ]:
# Diebold-Mariano 檢定
loss_diff = ar1_errors**2 - ar2_errors**2
dm_test = sm.OLS(loss_diff, np.ones(len(loss_diff)), missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags': 2})
print("\nDiebold-Mariano Test Result:")
print(dm_test.summary())

* (b)  解釋 `dm_test.summary()` 的結果：
        * `coef`：損失差異的平均值。
        * `P>|z|`：P 值，用於判斷是否拒絕原假設。
        * 其他統計量（如 z-statistic）用於判斷顯著性。
    * (c)  根據 DM 檢定的 P 值判斷。若 P 值小於顯著水準（例如 0.05），則拒絕原假設，認為兩個模型的預測能力有顯著差異；否則，不拒絕原假設，認為兩個模型的預測能力沒有顯著差異。結合 MSE 的結果，可以得出最終結論。

這個題組提供了一個完整的實作練習，涵蓋了資料處理、模型建立、預測能力評估和統計檢定。這將幫助學習者更深入地理解時間序列分析和模型評估的實際應用。